# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sinop,41.7711,34.8709,1.24,61,46,5.02,TR,1701066063
1,1,grytviken,-54.2811,-36.5092,0.99,88,61,5.85,GS,1701066619
2,2,palmer,42.1584,-72.3287,11.59,82,100,2.06,US,1701066619
3,3,ancud,-41.8697,-73.8203,9.29,85,100,1.59,CL,1701066619
4,4,stanley,54.8680,-1.6985,3.62,96,40,2.57,GB,1701066619


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.7,
    color = "City"
)

# Display the map plot
humidity_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,bandar-e genaveh,29.5791,50.5170,24.01,60,0,3.56,IR,1701066629
140,140,bubaque,11.2833,-15.8333,26.63,67,0,2.95,GW,1701066637
296,296,sur,22.5667,59.5289,26.71,44,0,2.71,OM,1701066660
345,345,manzanillo,20.3433,-77.1167,23.17,86,0,0.54,CU,1701066668
482,482,vieux-habitants,16.0589,-61.7659,22.76,68,0,3.11,GP,1701066688


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
75,bandar-e genaveh,IR,29.5791,50.5170,60,
140,bubaque,GW,11.2833,-15.8333,67,
296,sur,OM,22.5667,59.5289,44,
345,manzanillo,CU,20.3433,-77.1167,86,
482,vieux-habitants,GP,16.0589,-61.7659,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
       "categories":"accommodation.hotel",
        "apiKey": geoapify_key,
        "limit":1

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
                                
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bandar-e genaveh - nearest hotel: منزل دکتور سلمان
bubaque - nearest hotel: Chez Julio
sur - nearest hotel: Sur Hotel
manzanillo - nearest hotel: Hotel Venus
vieux-habitants - nearest hotel: Gîtes "Les Cocotiers"
calvinia - nearest hotel: Calvinia Hotel
ghormach - nearest hotel: No hotel found
saint croix - nearest hotel: The Buccaneer


,City,Country,Lat,Lng,Humidity,Hotel Name
75,bandar-e genaveh,IR,29.5791,50.5170,60,منزل دکتور سلمان
140,bubaque,GW,11.2833,-15.8333,67,Chez Julio
296,sur,OM,22.5667,59.5289,44,Sur Hotel
345,manzanillo,CU,20.3433,-77.1167,86,Hotel Venus
482,vieux-habitants,GP,16.0589,-61.7659,68,"Gîtes ""Les Cocotiers"""
497,calvinia,ZA,-31.4707,19.7760,31,Calvinia Hotel
515,ghormach,AF,35.7306,63.7826,25,No hotel found
527,saint croix,VI,17.7397,-64.7388,80,The Buccaneer


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plotay

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    hover_cols = ["Country","Hotel Name"]
)

# Display the map plot
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)